## Create CONSTANTS

In [1]:
DATASET = [
    {
        'Tid': 'T1',
        'Item': ['a', 'b', 'c', 'd'],
        'Quantity': [5,2,1,2]
    },
    {
        'Tid': 'T2',
        'Item': ['a', 'c', 'd', 'g'],
        'Quantity': [1,1,1,3]
    },
    {
        'Tid': 'T3',
        'Item': ['a', 'c', 'f'],
        'Quantity': [1,1,1]
    },
    {
        'Tid': 'T4',
        'Item': ['a', 'f', 'g'],
        'Quantity': [1,4,2]
    },
    {
        'Tid': 'T5',
        'Item': ['a', 'g'],
        'Quantity': [1,2]
    },
    {
        'Tid': 'T6',
        'Item': ['b', 'c', 'd', 'e'],
        'Quantity': [3,2,3,1]
    },
    {
        'Tid': 'T7',
        'Item': ['c', 'e'],
        'Quantity': [6,4]
    },
    {
        'Tid': 'T8',
        'Item': ['e', 'f'],
        'Quantity': [1,3]
    }
]
UNIT_UTILITY = {
    'a': 3,
    'b': 6,
    'c': -3,
    'd': 12,
    'e': -5,
    'f': -2,
    'g': -1
}

EPM = {
    'min_util': 30,
    'min_per': 1,
    'max_per': 5,
    'min_avg': 1,
    'max_avg': 3
}

### Show global variable

In [2]:
for i in DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['a', 'b', 'c', 'd'], 'Quantity': [5, 2, 1, 2]}
{'Tid': 'T2', 'Item': ['a', 'c', 'd', 'g'], 'Quantity': [1, 1, 1, 3]}
{'Tid': 'T3', 'Item': ['a', 'c', 'f'], 'Quantity': [1, 1, 1]}
{'Tid': 'T4', 'Item': ['a', 'f', 'g'], 'Quantity': [1, 4, 2]}
{'Tid': 'T5', 'Item': ['a', 'g'], 'Quantity': [1, 2]}
{'Tid': 'T6', 'Item': ['b', 'c', 'd', 'e'], 'Quantity': [3, 2, 3, 1]}
{'Tid': 'T7', 'Item': ['c', 'e'], 'Quantity': [6, 4]}
{'Tid': 'T8', 'Item': ['e', 'f'], 'Quantity': [1, 3]}


In [3]:
print(UNIT_UTILITY)

{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}


In [4]:
print(EPM)

{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [5]:
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # for i in dataset:
    #     print(i)
    # calculate TWU of each item in dataset
    twu_dict = dict()
    twu_transaction_dict = dict()
    ps_dict = dict()
    transaction_dict = dict()
    for i in dataset:
        twu_transaction_dict[i['Tid']] = 0
        for j in range(len(i['Item'])):
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []

            twu_transaction_dict[i['Tid']] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))

        for j in range(len(i['Item'])):
            twu_dict[i['Item'][j]] += twu_transaction_dict[i['Tid']]
    
    # print(twu_transaction_dict)
    for i in transaction_dict.keys():
        ps_dict[i].append(transaction_dict[i][0])
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])
    
    temp_dataset = list()
    eucs_dict = dict()

    for i in dataset:
        temp_item_quantity = list()
        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]])/len(ps_dict[i['Item'][j]])
            # print((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per']\
            and avg_item >= epm['min_avg']:
                
                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        temp_item_quantity.sort(key=lambda x: x[3])
        print(temp_item_quantity)
        temp_item_quantity.sort(key=lambda x: x[2])
        print(temp_item_quantity)
        # print(i)
        temp_dataset.append(
            {
                'Tid': i['Tid'],
                'Item': list(map(lambda x: x[0], temp_item_quantity)),
                'Quantity': list(map(lambda x: x[1], temp_item_quantity))
            }
        )
        # print(i, end='\n\n')
    
    for i in temp_dataset:
        for j in range(len(i['Item']) - 1):
            if i['Item'][j] + i['Item'][j + 1] not in eucs_dict:
                eucs_dict[i['Item'][j] + i['Item'][j + 1]] = 0
            eucs_dict[i['Item'][j] + i['Item'][j + 1]] += twu_transaction_dict[i['Tid']]
    
    return temp_dataset, eucs_dict

In [6]:
NEW_DATASET, EUCS_LIST = create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

[('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120)]
[('a', 1, 3, 75), ('d', 1, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75), ('d', 1, 12, 120)]
[('a', 1, 3, 75), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('b', 3, 6, 105), ('d', 3, 12, 120), ('c', 2, -3, 123)]
[('c', 2, -3, 123), ('b', 3, 6, 105), ('d', 3, 12, 120)]
[('c', 6, -3, 123)]
[('c', 6, -3, 123)]
[]
[]


In [7]:
for i in NEW_DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['c', 'a', 'b', 'd'], 'Quantity': [1, 5, 2, 2]}
{'Tid': 'T2', 'Item': ['c', 'a', 'd'], 'Quantity': [1, 1, 1]}
{'Tid': 'T3', 'Item': ['c', 'a'], 'Quantity': [1, 1]}
{'Tid': 'T4', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T5', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T6', 'Item': ['c', 'b', 'd'], 'Quantity': [2, 3, 3]}
{'Tid': 'T7', 'Item': ['c'], 'Quantity': [6]}
{'Tid': 'T8', 'Item': [], 'Quantity': []}


In [8]:
print(EUCS_LIST)

{'ca': 69, 'ab': 51, 'bd': 105, 'ad': 15, 'cb': 54}


## Searching stage

In [ ]:
def construct(prefix: dict, x: dict, y: dict) -> dict:

    result = dict()
    for i in x.keys():
        if i in y:
            result[i] = (x[i][0] + y[i][0] - prefix[i][0], min(x[i][1], y[i][1]))
    return result

def utility_condition(x, minutil):
    return sum(x[i][0] for i in x.keys()) >= minutil

def calculate_periodic(x):
    periodic = list()
    keys = x.keys()
    for i in range(len(keys) - 1):
        periodic.append(int(keys[i][1:]) - int(keys[i + 1][1:]))

    return max(periodic), min(periodic), sum(periodic) / len(periodic)
def periodic_condition(x, minper, maxper, minavg, maxavg):
    max_i, min_i, avg_i = calculate_periodic(x)
    return min_i >= minper and max_i <= maxper and avg_i >= minavg and avg_i <= maxavg

def ru_condition(x, minutil):
    return sum(x[i][0] + x[i][1] for i in x.keys()) >= minutil

def max_per_condition(x, maxper):
    max_x, min_x, avg_x = calculate_periodic(x)
    return max_x <= maxper

def avg_per_condition(x, maxavg):
    max_x, min_x, avg_x = calculate_periodic(x)
    return avg_x <= maxavg

def eucp_condition(X, Y, eucs_list, minutil):
    items_name = X + Y if X + Y in eucs_list else Y + X
    return eucs_list[items_name] >= minutil

In [ ]:
def phm_seach(prefix : str, prefix_list: dict, lists: dict) -> set:
    result = set()
    key_lists = lists.keys()
    for i in key_lists:
        if utility_condition(lists[i]) and periodic_condition(lists[i]):
            result.add(i)
        if ru_condition(lists[i]) and max_per_condition(lists[i]) and avg_per_condition(lists[i]):
            new_list = dict()

            for j in key_lists[key_lists.index(i) + 1:]:
                if eucp_condition(i,j):
                    z = i + j[-1]
                    z_list = construct(prefix, i, j)
                    new_list[z] = z_list
            result.update(phm_seach(i, lists[i], new_list))
            
    return result

In [ ]:
def search(prefix : str, prefix_list: list, lists: list, item_utility: list) -> set:
    result = set()
    for i in lists:
        if utility_condition(i) and periodic_condition(i):
            result.add(i)
        elif ru_condition(i) and max_per_condition(i) and avg_per_condition(i):
            new_list = list()

            for j in lists[lists.index(i) + 1:]:
                if eucp_conditon(i,j):
                    compute_du(j,i)
                    z = i + j # wrong fix it
                    if isinstance(j, m_list):
                        z_list = construct(,i,)
